In [4]:
import json
import boto3
import time

## Transcribe Audio

In [20]:
job_name = "test3"
job_uri = 'https://s3.amazonaws.com/callhome.audio.dataset/data/mp3/4074.mp3'
output_bucket = 'random.datasets.sengstacken'
role = 'arn:aws:iam::431615879134:role/service-role/AmazonTranscribeServiceRoleFullAccess-MyTranscribeRole'

In [21]:
# transcribe audio
transcribe = boto3.client('transcribe')

response = transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    LanguageCode='en-US',
    Media={
        'MediaFileUri':job_uri
    },
    OutputBucketName=output_bucket,
    Settings={
        'ChannelIdentification':True,
        'ShowAlternatives':True,
        'MaxAlternatives':2
    },
    JobExecutionSettings={
        'AllowDeferredExecution': True,
        'DataAccessRoleArn':role
    }
)

while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        print(status['TranscriptionJob']['TranscriptionJobStatus'])
        break
    print("Not done yet!")
    time.sleep(5)
    
print(transcribe.get_transcription_job(TranscriptionJobName=job_name))

Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
Not done yet!
{'TranscriptionJob': {'TranscriptionJobName': 'test3', 'Transc

In [22]:
print(transcribe.get_transcription_job(TranscriptionJobName=job_name))

{'TranscriptionJob': {'TranscriptionJobName': 'test3', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'en-US', 'MediaSampleRateHertz': 8000, 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 'https://s3.amazonaws.com/callhome.audio.dataset/data/mp3/4074.mp3'}, 'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/random.datasets.sengstacken/test3.json'}, 'StartTime': datetime.datetime(2020, 9, 25, 14, 35, 18, 56000, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2020, 9, 25, 14, 35, 18, 33000, tzinfo=tzlocal()), 'CompletionTime': datetime.datetime(2020, 9, 25, 14, 40, 54, 58000, tzinfo=tzlocal()), 'Settings': {'ChannelIdentification': True, 'ShowAlternatives': True, 'MaxAlternatives': 2}}, 'ResponseMetadata': {'RequestId': 'cc8e5fb7-cb6f-4449-bcce-1bb528aed6c0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 25 Sep 2020 14:40:57 GMT', 'x-amzn-requestid': 'cc8e5fb7-cb6f-4449-bcce-1bb528aed6c0', 'content-le

In [23]:
transcript = transcribe.get_transcription_job(TranscriptionJobName=job_name)['TranscriptionJob']['Transcript']['TranscriptFileUri']

## Get Transcription from s3

In [24]:
transcript

'https://s3.us-east-1.amazonaws.com/random.datasets.sengstacken/test3.json'

In [25]:
s3 = boto3.resource('s3')

In [6]:
transcript = 's3://callhome.audio.dataset/output/json/call-home_2020_03_02_17_40_24_4074.mp3.json'

In [26]:
bucket = transcript.split('/')[3]
key = '/'.join(transcript.split('/')[4:])

In [27]:
obj = s3.Object(bucket, key)

In [28]:
file_content = obj.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
raw_transcript = json_content['results']['transcripts'][0]['transcript']

In [29]:
json_content

{'jobName': 'test3',
 'accountId': '431615879134',
 'results': {'transcripts': [{'transcript': "So did you just hear that message? No. No, I don't have anything, right? I actually have to call someone who has an American accent who is living outside of the United States because of the thing that what they want is, they want American accents, uh, Yes. Yes, I did, but no, no, no. I don't have a British accent or anything. Okay? Oh, all right. Fair enough. for their database. I also could have, uh, gone if I was a native Spanish speaker or native camel speaker or, you know, they have this whole list of languages, but both people on the line have to have the same No. native speech. So, Alright. Sounds fair. uh, yeah, it's kind of an odd thing, but in any case, you know, like I said, So? So So So So So do you have to say anything in particular or just or do they just want So So So you have to say anything in particular, or just I just had to get your permission to record it. Oh, okay. And a

## Pretty Formatter

In [30]:
def format_transcript(json_transcript_uri):
    
    import json
    import datetime
    
    temp = []
    s3 = boto3.resource('s3')
    bucket = json_transcript_uri.split('/')[3]
    key = '/'.join(json_transcript_uri.split('/')[4:])
    
    # read file
    obj = s3.Object(bucket, key)
    file_content = obj.get()['Body'].read().decode('utf-8')
    data = json.loads(file_content)
    raw_transcript = json_content['results']['transcripts'][0]['transcript']
    
    with open('temp.txt','w') as w:
        
        # check type of transcript - channel vs speaker 
        if 'channel_labels' in json_content['results'].keys():
            labels = data['results']['channel_labels']['channels']
            var = 'channel_label'

        elif 'speaker_labels' in json_content['results'].keys():
            labels = data['results']['speaker_labels']['speakers']
            val = 'speaker_label'

        else:
            'Not channel or speaker encoded'
            
        # get speaker times
        speaker_start_times={}
        for label in labels:
            print
            for item in label['items']:
                start_time = item.get("start_time", "")
                if start_time != "":
                    speaker_start_times[item['start_time']] = label[var]
                    
        # get text per speaker
        items = data['results']['items']
        lines, line, time = [], '', 0
        speaker, i = 'null', 0
        for item in items:
            i=i+1
            content = item['alternatives'][0]['content']
            if item.get('start_time'):
                current_speaker=speaker_start_times[item['start_time']]
            elif item['type'] == 'punctuation':
                line = line+content
            if current_speaker != speaker:
                if speaker:
                    lines.append({'speaker':speaker, 'line':line, 'time':time})
                line=content
                speaker=current_speaker
                time=item['start_time']
            elif item['type'] != 'punctuation':
                line = line + ' ' + content
        lines.append({'speaker':speaker, 'line':line,'time':time})
        
        # output to file
        sorted_lines = sorted(lines,key=lambda k: float(k['time']))
        for line_data in sorted_lines:
            line='[' + str(datetime.timedelta(seconds=int(round(float(line_data['time']))))) + '] ' + line_data.get('speaker') + ': ' + line_data.get('line')
            w.write(line + '\n\n')
    
    return sorted_lines

In [31]:
%%time
tran_list = format_transcript(transcript)

CPU times: user 84.2 ms, sys: 25 ms, total: 109 ms
Wall time: 201 ms


## Extract Entities and Phrases with Comprehend

In [32]:
def get_named_entities_n_phrases(json_transcript_uri):
    # function to get named entities and key phrases from comprehend
    
    import json
    import datetime
    
    lang = 'en'
    s3 = boto3.resource('s3')
    bucket = json_transcript_uri.split('/')[3]
    key = '/'.join(json_transcript_uri.split('/')[4:])
    
    # read file
    obj = s3.Object(bucket, key)
    file_content = obj.get()['Body'].read().decode('utf-8')
    data = json.loads(file_content)
    transcript = json_content['results']['transcripts'][0]['transcript']
    
    comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
    
    # check the size of the transcript, if needed chunk the data
    if len(transcript.encode('utf-8')) > 5000:
        chunk, counter = 4900,0
        temp, temp2 = [], []

        while chunk*counter < len(transcript.encode('utf-8')):
            tchunk = transcript[counter*chunk:(counter+1)*chunk]
            ent = comprehend.detect_entities(Text=tchunk,LanguageCode=lang) 
            temp = temp + ent['Entities']
            kp = comprehend.detect_key_phrases(Text=tchunk,LanguageCode=lang) 
            temp2 = temp2 + kp['KeyPhrases']
            counter+=1
            
    else:
        ent = comprehend.detect_entities(Text=transcript,LanguageCode=lang)
        temp = ent['Entities']
        kp = comprehend.detect_key_phrases(Text=transcript,LanguageCode=lang)
        temp2 = kp['KeyPhrases']

    res_dict = {"Entities": temp, "KeyPhrases": temp2}

    return res_dict

In [33]:
transcript

'https://s3.us-east-1.amazonaws.com/random.datasets.sengstacken/test3.json'

In [34]:
ent_and_phrases = get_named_entities_n_phrases(transcript)

In [36]:
ent_and_phrases['Entities']

[{'Score': 0.8172644972801208,
  'Type': 'OTHER',
  'Text': 'American',
  'BeginOffset': 116,
  'EndOffset': 124},
 {'Score': 0.9991639852523804,
  'Type': 'LOCATION',
  'Text': 'United States',
  'BeginOffset': 161,
  'EndOffset': 174},
 {'Score': 0.8515924215316772,
  'Type': 'OTHER',
  'Text': 'American',
  'BeginOffset': 230,
  'EndOffset': 238},
 {'Score': 0.9223531484603882,
  'Type': 'OTHER',
  'Text': 'British',
  'BeginOffset': 300,
  'EndOffset': 307},
 {'Score': 0.9215112328529358,
  'Type': 'OTHER',
  'Text': 'Spanish',
  'BeginOffset': 428,
  'EndOffset': 435},
 {'Score': 0.9283924698829651,
  'Type': 'QUANTITY',
  'Text': 'both people',
  'BeginOffset': 526,
  'EndOffset': 537},
 {'Score': 0.8815425038337708,
  'Type': 'QUANTITY',
  'Text': 'three different programs',
  'BeginOffset': 1140,
  'EndOffset': 1164},
 {'Score': 0.9748173952102661,
  'Type': 'QUANTITY',
  'Text': 'one',
  'BeginOffset': 1181,
  'EndOffset': 1184},
 {'Score': 0.5994881391525269,
  'Type': 'QUANT

In [41]:
entity_list = [a['Text'] for a in ent_and_phrases['Entities'] if a['Type'] in ['PERSON','OTHER','DATE','LOCATION','ORGANIZATION']] # excludes 'QUANTITY'

In [42]:
entity_list

['American',
 'United States',
 'American',
 'British',
 'Spanish',
 'United States',
 'United States',
 'Japan',
 'Japan',
 '800',
 'yesterday',
 '800',
 'Chris',
 'Chris',
 'next 23 years',
 'next 23 years',
 'this year',
 'this year',
 'Adrian',
 'this year',
 'this year',
 'May',
 'France',
 'Neil',
 'Neil',
 'Rochester',
 'Neil',
 'a week later',
 'Branmar',
 'this year',
 'last year',
 'l Carson',
 'US',
 'U. S.',
 'U. S.',
 'Chris',
 'Chris',
 'Chris',
 'Nelson',
 'Nelson',
 'Nelson',
 'Europe',
 'Europe',
 'Italy',
 'England',
 'Chris',
 'Chris',
 'Branmar',
 'Branmar',
 'Harvard',
 'England',
 'Harvard',
 'England',
 'Einstein',
 'Einstein',
 'San',
 'san',
 'Switzerland',
 'few months ago',
 'Switzerland',
 'few months ago',
 'Einstein',
 'Einstein',
 'Roger',
 'Penrose',
 'Eric',
 'Florida',
 'Florida',
 'Frohlich',
 'Cartesian',
 'Newtonian',
 'Newtonian',
 'Dracula',
 'June 5th',
 'Joe',
 'Susan',
 'Frank',
 'June 5th',
 'Joe',
 'Susan',
 'Frank',
 'Bram Stoker',
 'London'

In [43]:
entity_string = ', '.join(entity_list)

In [44]:
entity_string

'American, United States, American, British, Spanish, United States, United States, Japan, Japan, 800, yesterday, 800, Chris, Chris, next 23 years, next 23 years, this year, this year, Adrian, this year, this year, May, France, Neil, Neil, Rochester, Neil, a week later, Branmar, this year, last year, l Carson, US, U. S., U. S., Chris, Chris, Chris, Nelson, Nelson, Nelson, Europe, Europe, Italy, England, Chris, Chris, Branmar, Branmar, Harvard, England, Harvard, England, Einstein, Einstein, San, san, Switzerland, few months ago, Switzerland, few months ago, Einstein, Einstein, Roger, Penrose, Eric, Florida, Florida, Frohlich, Cartesian, Newtonian, Newtonian, Dracula, June 5th, Joe, Susan, Frank, June 5th, Joe, Susan, Frank, Bram Stoker, London, English, Christian'

## Extract Sentiment Per Utterance

In [45]:
def detect_sentiment_per_utterance(data):
    
    # function to get sentiment per utterance.  
    # Assumes that the input data stream is a list of dictionaries.  
    # The dictionary should have a 'line' field with text
    
    comprehend = boto3.client('comprehend')
    for l in data:
        if l['line'] != '':
            temp = comprehend.detect_sentiment(Text=l['line'], LanguageCode='en')
            l['Sentiment'] = temp['Sentiment']
            l['SentimentScore'] = temp['SentimentScore']
            
    return data

In [46]:
%%time
tran_list = detect_sentiment_per_utterance(tran_list)

CPU times: user 2.89 s, sys: 100 ms, total: 2.99 s
Wall time: 52.4 s


In [48]:
tran_list

[{'speaker': 'null', 'line': '', 'time': 0},
 {'speaker': 'ch_0',
  'line': 'So did you just hear that message? No.',
  'time': '1.54',
  'Sentiment': 'NEUTRAL',
  'SentimentScore': {'Positive': 0.0035443566739559174,
   'Negative': 0.2406766414642334,
   'Neutral': 0.7557653188705444,
   'Mixed': 1.3739350833930075e-05}},
 {'speaker': 'ch_1',
  'line': "Yes. Yes, I did, but no, no, no. I don't have a British accent or anything. Okay? Oh, all right. Fair enough.",
  'time': '3.24',
  'Sentiment': 'NEUTRAL',
  'SentimentScore': {'Positive': 0.22071564197540283,
   'Negative': 0.264838844537735,
   'Neutral': 0.5132074952125549,
   'Mixed': 0.0012379923136904836}},
 {'speaker': 'ch_1',
  'line': "No, I don't have",
  'time': '4.84',
  'Sentiment': 'NEUTRAL',
  'SentimentScore': {'Positive': 0.016288023442029953,
   'Negative': 0.44253623485565186,
   'Neutral': 0.5411473512649536,
   'Mixed': 2.8330046916380525e-05}},
 {'speaker': 'ch_0',
  'line': 'anything, right? I actually have to ca

In [ ]:
# pretty transcript

In [53]:
import datetime
with open('temp_w_sentiment.txt','w') as w:
    
    for line_data in tran_list:
        if line_data.get('Sentiment') != None:
            line='[' + str(datetime.timedelta(seconds=int(round(float(line_data['time']))))) + '] ' + line_data.get('speaker') + ' - '  + line_data.get('Sentiment') + ': '  + line_data.get('line')
            w.write(line + '\n\n')

## Plot Sentiment over Time - IN WORK

In [119]:
list(set([a['speaker'] for a in tran_list]))

['null', 'ch_1', 'ch_0']

In [129]:
# get list of all speakers / channels
speakers = list(set([a['speaker'] for a in tran_list]))

for speaker in speakers:
    times = []
    val = []
    if speaker != 'null':
        for l in tran_list:
            if l['line'] != '':
                if l['speaker'] == speaker:
                    times.append(l['time'])
                    if l['Sentiment'] == 'NEUTRAL':
                        val.append(0.)
                    elif l['Sentiment'] == 'NEGATIVE':
                        val.append(-1.)
                    elif l['Sentiment'] == 'POSITIVE':
                        val.append(1.)
                    else:
                        val.append(0.)

In [ ]:
def plot_sentiment_by_time(data):
    
    # get list of all speakers / channels
    speakers = list(set([a['speaker'] for a in tran_list]))

    for speaker in speakers:
        times = []
        val = []
        if speaker != 'null':
            for l in data:
                if l['line'] != '':
                    if l['speaker'] == speaker:
                        times.append(l['time'])
                        val.append(l['Sentiment'])
                        
                    
                    
                    
            
            
    